# Прогноз стоимости автомобилей
## Задача: спрогнозировать стоимость автомобилей, используя данные с сайта www.auto.ru

In [1]:
# Импортируем библиотеки

import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

from sklearn.ensemble import StackingRegressor

from sklearn.base import clone
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import KFold

from datetime import datetime

from catboost import CatBoostRegressor
from sklearn.ensemble import (AdaBoostClassifier, GradientBoostingClassifier,
                              RandomForestClassifier, ExtraTreesClassifier)
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor, RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

import lightgbm as lgb
from lightgbm import LGBMRegressor

In [2]:
test = pd.read_csv('../input/sf-dst-car-price-prediction/test.csv')
train = pd.read_csv('../input/avto-df/avto_all.csv')
sample_submission = pd.read_csv('../input/sf-dst-car-price-prediction/sample_submission.csv')

In [3]:
test.sample(3)

In [4]:
test.info()

In [5]:
train.sample(3)

In [6]:
train.info()

In [7]:
# В test 31 колонка, а в train 21. Приведем все к общему количеству и переименуем названия
train.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'title', 'times', 'views', 'tax'], axis=1, inplace=True)
test.drop(['car_url', 'complectation_dict', 'description', 'equipment_dict', 'modelDate', 'model_info', 'image', 'parsing_unixtime', 'priceCurrency', 'super_gen', 'vendor', 'Владение', 'vehicleConfiguration', 'name', 'numberOfDoors'], axis=1, inplace=True)

In [8]:
test.sample(3)
test.info()

In [9]:
# Переименуем столбы

test = test.rename({'bodyType': 'body_type', 'Владельцы': 'onwers_count', 'ПТС': 'pts', 'Привод': 'drive', 'Руль': 'wheel', 'Состояние': 'state', 'Таможня': 'castoms', 'sell_id': 'id', 'mileage':'kmage', 'productionDate': 'year', 'vehicleTransmission': 'transmision', 'productionDate': 'year'}, axis=1)

In [10]:
# в test столбец ПТС содержит одно значение nan, заполним его наиболее часто встречающимся значением.
test.isna().sum()

In [11]:
#test.dropna(inplace=True)
test['pts'].unique()
#test[test['pts']== 'NaN']
test.groupby(['pts']).head()

In [12]:
#test.index([10412])
test.at[10412, 'pts'] = 'Оригинал'

In [13]:
# Найдем пустые значения
train.isnull().sum()

In [14]:
train[train['year'] == 'Nan']

In [15]:
train.dropna(subset=['year'], inplace=True)

In [16]:
# Сравним название моделей машин в test и train
test['brand'].unique()

In [17]:
train['brand'].unique()

In [18]:
# train содержит различное количество моделей автомобилей, в test присутствует только 12, поэтому создадим датасет содержащий модели test. Далее приведем все названия к одному формату. 

df_brand = train.loc[train['brand'].isin(['Infiniti', 'Skoda', 'Mercedes-Benz', 'Volvo','BMW', 'Toyota', 'Lexus', 'Volkswagen', 'Mitsubishi', 'Audi', 'Honda', 'Nissan'])]
mapping = {'Infiniti':'INFINITI','Skoda':'SKODA','Mercedes-Benz':'MERCEDES', 'Audi':'AUDI', 'Honda': 'HONDA', 'Nissan': 'NISSAN', 'Volvo':'VOLVO','Toyota':'TOYOTA', 'Lexus':'LEXUS', 'Volkswagen':'VOLKSWAGEN', 'Mitsubishi':'MITSUBISHI', 'BMW':'BMW'}
df_brand['brand'] = df_brand['brand'].map(mapping)

In [19]:
# Удалим дубликаты

df_brand.drop_duplicates(subset=['id'], keep='last', inplace=True)

In [20]:
# Посмотрим сколько осталось значений
df_brand.info()

In [21]:
# Посмотрим на количество моделей в датасете
df_brand['brand'].value_counts()

In [22]:
df_brand.columns.sort_values()

In [23]:
#test.columns.sort_values()

In [24]:
# добавим в test столбец price и заполним его nan

test['price'] = np.nan
df_brand['sample'] = 1
test['sample'] = 0

In [25]:
# преобразуем столбец с пробегом в число, откинем обозначение "км"

df_brand['kmage'] = df_brand['kmage'].apply(lambda x: x[:-2])

#df_brand['kmage_'] = df_brand['kmage'].str.extract(r'(\d+\.\d+|\d+)', expand=False)
#df_brand['kmage_'] = df_brand['kmage'].apply(lambda x: x.replace(' ', ''))
#df_brand['kmage_'] = df_brand['kmage_'].str.extract(r'(\d+\.\d+|\d+)', expand=False)
##df_brand['kmage_'] = df_brand['kmage_'].apply(lambda x: x.replace('\xa0', '').replace('N', ''))
#df_brand['kmage_'] = df_brand['kmage_'].apply(lambda x: int(x))
#df_brand["kmage_"] = pd.to_numeric(df_brand["kmage_"])
#df_brand["kmage_"] = df_brand["kmage_"].astype(int)
df_brand['kmage'] = pd.to_numeric(df_brand['kmage'].str.replace('\D', ''), errors='coerce')
#df_brand['kmage_'] = df_brand['kmage_'].apply(lambda x: int(x) if x.isdigit() else x)
#df_brand['kmage_'] = df_brand['kmage'].str.replace('', ' ')
df_brand['kmage'].unique()
#df_brand.drop(['kmage_'], axis=1, inplace=True)

In [26]:
# Появились 34 пустых значения, выведим их
df_brand['kmage'].isna().sum()
#train[train['id'] == 1103448349]

In [27]:
df_brand[df_brand['kmage'].isna()]

In [28]:
# Удалим строки с пустыми значениями
df_brand.dropna(subset=['kmage'], inplace=True)

In [29]:
#df_brand[df_brand['kmage'].isnull()]
#df_brand[df_brand['id'] == 1104634191]
df_brand.isnull().sum()

In [30]:
# Удалим пустые значения
df_brand.dropna(inplace=True)

In [31]:
# Объединим датафреймы
data = pd.concat([df_brand, test])
data.sample(3)

In [32]:
# проверим датасет на наличие дубликаов и удалим их

data = data.drop_duplicates(['id'], keep='last')

In [33]:
# Столбец в ценой имеет нулевые значения, т.к. test у нас был без цены. Также у нас отсутствует одно значение в ПТС. Удалим его
data.info()

In [34]:
#data[data['pts'].isna()]

In [35]:
#data.dropna(subset=['pts'], inplace=True)

EDA


После удаления отличающихся признаков у нас остались следующие признаки:


id - id объявления,

brand - марка автомобиля,

title - полное название модели автомобиля,

year - дата производства автомобиля,

kmage - пробег,

body_type - тип кузова,

color - цвет автомобиля,

engineDisplacement - объём двигателя,

enginePower - мощность двигателя,

fuelType - тип топлива,

transmision - трансмиссия,

drive - тип привода,

wheel - сторона руля,

state - состояние,

owners_count - количество владельцев,

pts - наличие ПТС,

customs - этап растаможки,

price - цена автомобиля,

sample - индикатор принадлежности данных к тесту (0) и трейну (1).

In [36]:
# Создадит списки для категориальных и числовых признаков:

cat = []
num = []
bin = []

## 1. id оставляем как есть

## 2. brand мы обрабатывали выше, до соединеия датасетов

In [37]:
data['brand'].unique()
cat.append('brand')

In [38]:
# Столбец с годом выпуска преобразуем в числовой формат
data['year'] = pd.to_numeric(data['year'])
data['year'].hist()

In [39]:
num.append('year')

In [40]:
# Пробег авто мы обработали ранее, он у нас должен состоять их числовых значений. Средний пробег автомобилей 150т.км
data['kmage'].mean()

In [41]:
num.append('kmage')

In [42]:
# body_type - Тип кузова авто. С train у нас еще подтянулось количество дверей. Приведем признак к категориальному

data['body_type'].unique()
data[data['body_type'] == 'Nan']

In [43]:
# Удалим значение имеющее тип кузова - Nan
data.drop(index = [52704], inplace=True)

In [44]:
# В основном авто с типом кузова седан и внедорожник
data['body_type'].value_counts().plot.barh()

In [45]:
cat.append('body_type')

In [46]:
# В 16 цветах представлены автомобили, больше всего черного цвета.

data['color'].nunique()

In [47]:
data['color'].value_counts().plot.barh()

In [48]:
cat.append('color')

In [49]:
# Столбец с обемом двигателя имеет различные значения, к тому у электромобилей нет этого значения и они заполнились мощностью двигателя
# Не понятно чем можно заполнить значение обьъма двигателя у электромобилей, так как у нас есть столбец с мощностью двигателя, удалим данный столбец
data['engineDisplacement'].unique()

In [50]:
data.drop(['engineDisplacement'], axis=1, inplace=True)

In [51]:
# Мощность авто также представлена в различных вариантах 
data['enginePower'].unique()

In [52]:

data['enginePower'] = data['enginePower'].apply(lambda x: x.replace(' ', ''))
#data['enginePower_'] = data['enginePower'].apply(lambda x: x[:4])
#data['enginePower_'] = data['enginePower_'].apply(lambda x: x.replace('\xa0', '').replace(' ', ''))
data['enginePower'] = data['enginePower'].str.extract(r'(\d+\.\d+|\d+)', expand=False)
data['enginePower'] = pd.to_numeric(data['enginePower'], errors='coerce')
#data['enginePower_'] = pd.to_numeric(data['enginePower_'], errors='coerce')
#data['enginePower_'] = data['enginePower_'].apply(filter(lambda x: x.isdigit))
#data['enginePower'].sum()
data['enginePower'].plot()

In [53]:
# Посмотрим на максимальную и минимальную мощность двигателя
# посмотрев в интернете, действительно был такой автомобиль марки SKODA с двигателем 30лс
# Приведем признак к категориальному, группируя мощность двигателя по 100лс к каждой категории

print('мин. мощность двигателя =', data['enginePower'].min())
print('макс. мощность двигателя =', data['enginePower'].max())

In [54]:
def enginepower(x):
    if x <= 100: x = 1
    elif 100 < x <= 200: x = 2
    elif 200 < x <= 300: x = 3
    elif 300 < x <= 400: x = 4
    elif 400 < x <= 500: x = 5
    elif 500 < x <= 600: x = 6
    elif 600 < x < 700: x = 7
    else: x = 8
    return x
data['Power'] = data['enginePower'].apply(lambda x: enginepower(x))
data[['Power', 'enginePower']]

In [55]:
cat.append('Power')

In [56]:
# Значения по виду топлива приведем к 4 категориям:
# - бензин
# - дизель
# - газ
# - гибрид

data['fuelType'].unique()

In [57]:
fuel_dict = {' Дизель':'дизель', 
                       ' Бензин':'бензин',
                       'бензин': 'бензин',
                       ' Бензин, газобаллонное оборудование':'газ', 
                       ' Гибрид':'гибрид',
                       'гибрид': 'гибрид',
                       ' Газ, газобаллонное оборудование':'газ',
                       ' Дизель, газобаллонное оборудование':'газ', 
                       'дизель':'дизель',
                        'электро': 'электро',
                        ' Электро': 'электро',
                        'Газ': 'газ',
                        'газ':'газ'
                        }
data['fuelType'] = data['fuelType'].map(fuel_dict)
data['fuelType'].sample(3)
data['fuelType'].unique()

In [58]:
# Удалим строки в которых есть хотябы один Nan
data.dropna(subset=['fuelType'], inplace=True)
#data.dropna(how='any',axis=0, inplace=True)

In [59]:
# Большинство автомобилей представлено на бензине
plt.figure(figsize=(16,8))
sns.countplot(x = data['fuelType'], data = data) 

In [60]:
cat.append('fuelType')

In [61]:
# Тип коробки представленв 4 модификациях. Наибольшее количество авто с автоматической коробкой
data['transmision'].unique()

In [62]:
# Тут все значения чистые, обрабатывать ничего не надо. Основная часть авто с автоматической коробкой
plt.figure(figsize=(16,8))
sns.countplot(x = data['transmision'], data = data) 

In [63]:
cat.append('transmision')

In [64]:
# Тип привода представлен в 3 вариантах, больше всего машин с полным и передним приводом

data['drive'].unique()

In [65]:
plt.figure(figsize=(16,8))
sns.countplot(x = data['drive'], data = data) 

In [66]:
cat.append('drive')

In [67]:
# Тип расположения руля в основном левый

plt.figure(figsize=(16,8))
sns.countplot(x = data['wheel'], data = data)
bin.append('wheel') 

In [68]:
# Все автомобили представлены в одной категории
data['state'].unique()

In [69]:
# Находим значение в котором состояние авто NaN
data[data['state'] == 'Nan']
#data['state'].isna().sum()

In [70]:
# Заполним пропущенное значение состояния максимальным
#data['state'] = data['state'].fillna('111', inplace = True)
data.at[43735, 'state'] = 'Не требует ремонта'

In [71]:
# Количество владельцев в 3 категориях.
data['onwers_count'].unique()
data['onwers_count'] = data['onwers_count'].str.extract(r'(\d+\.\d+|\d+)', expand=False)
data['onwers_count'] = pd.to_numeric(data['onwers_count'], errors='coerce')
data['onwers_count'].unique()

In [72]:
# Найдем значение вк отором количество владельцев стоит NaN. Удалим его
data.groupby(['onwers_count']).head()
#data['onwers_count'].nunique()
#data[data['onwers_count'] == 'NaN']

In [73]:
data.drop(index = [98093], inplace=True)

In [74]:
plt.figure(figsize=(16,8))
sns.countplot(x = data['onwers_count'], data = data) 

In [75]:
cat.append('onwers_count')

In [76]:
# по натичию ПТС, 2 варианта, оригинал и дубликат. В основном это оригинал
data['pts'].unique()
plt.figure(figsize=(16,8))
sns.countplot(x = data['pts'], data = data) 

bin.append('pts')

In [77]:
data['castoms'].unique()
#plt.figure(figsize=(16,8))
#sns.countplot(x = data['castoms'], data = data) 

In [78]:
# Имеется автомобиль, где не указан признак castoms. Заполним его   
data[data['castoms'] == 'Nan']
#data['castoms'] = data['castoms'].fillna(max, inplace=True)

In [79]:
data.at[42701, 'castoms'] = 'Растаможен'

In [80]:
#data['price_'] = data['price'].str.extract(r'(\d+\.\d+|\d+)', expand=False)
data['price'] = pd.to_numeric(data['price'].str.replace('\D', ''), errors='coerce')
#data['price_'] = data['price_'].isna() == False
data['price']

In [81]:
# Информацию о состоянии авто и таможне можно удалить, т.к. все ячейки принимают одно значение
# Удалим также название модели
data.drop(['castoms', 'state', 'model_name'], axis=1, inplace=True) 

In [82]:
# Попробуем добавить новый признак, пробег за год. Для этого разделим пробег авто на его возраст
data['km_year'] = data['kmage'] / data['year']
num.append('km_year')

In [83]:
data.info()

In [84]:

# Создадим копию data, чтобы можно было легко к данному шагу вернуться и не обрабатывать все признаки заново
data_copy = data.copy()

In [85]:
#bin.remove('pts')
print(bin)
print(cat)
print(num)

## Числовые признаки

In [86]:
# Видна корреляция между данными признаками, но это и ожидаемо, т.к. чем старше машина, тем больше ее пробег
plt.figure(figsize=(10,5))
sns.heatmap(data[num].corr(), annot=True)

In [87]:
# Признаки можно попробовать нормализовать, т.к. имеется смещение
for i in num:
    plt.figure()
    sns.distplot(data[i][data[i] > 0].dropna(), kde = False, rug=False)
    plt.title(i)
    plt.show()

In [88]:
# После нормализации
for i in num:
    plt.figure()
    sns.distplot(np.log(data[i])[data[i] > 0].dropna(), kde = False, rug=False)
    plt.title(i)
    plt.show()

In [89]:
# Посмотрим на значимость непрерывных переменных:
# Наиболее значимым является год выпуска авто.
from pandas import Series, DataFrame

imp_num = Series(f_classif(data[data['price'].isna() == False][num], 
                           data[data['price'].isna() == False]['price'])[0], 
                           index = num)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

## Бинарные признаки

In [90]:
# Заменим значения бинарных признаков на 0 и 1:

label_encoder = LabelEncoder()
for i in bin:
    data[i] = label_encoder.fit_transform(data[i])
    
data[bin].head(5)

In [91]:
# Бинарные признаки имеют примерно обинаковую значимость
imp_cat = Series(mutual_info_classif(data[data['price'].isna() == False][bin], 
                                     data[data['price'].isna() == False]['price'],
                                     discrete_features=True), index=bin)
imp_cat.sort_values(inplace=True)
imp_cat.plot(kind='barh')

## Категориальные признаки

In [92]:
# Преобразуем все значения категориальных признаков в числа:
for i in cat:
    label_encoder.fit(data[i])
    data[i] = label_encoder.transform(data[i])

In [93]:
# Посмотрим на значимость категориальных признаков:
# видно, что марка авто является наиболее значимым признаком

imp_cat = Series(mutual_info_classif(data[data['price'].isna() == False][cat], 
                                     data[data['price'].isna() == False]['price'],
                                     discrete_features = True), index = cat)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

In [94]:
sns.heatmap(data[cat].corr().abs(), vmin=0, vmax=1, annot=True)

Наибольшая корреляция замента между типом кузова и приводом автомобиля

In [95]:
# Посмотрим значимость всех переменных на одном графике:
imp_cat = Series(mutual_info_classif(data[data['price'].isna() == False][cat + num + bin], 
                                     data[data['price'].isna() == False]['price'],
                                     discrete_features = True), index = cat + num + bin)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

Как видно из графика наибольшую значимость имеет пробег автомобиля в год

Корелляция между всеми признаками 

In [96]:
sns.heatmap(data[cat + num + bin].corr().abs(), vmin=0, vmax=1, annot=True)

Тут нового мы ничего не увидели. сильная корреляция заметна между годом выпуска авто и пробегом, а также между типом кузова и приводом 

In [97]:
 #Переведём категориальные признаки в dummy переменные:
data_copy = pd.get_dummies(data_copy, columns=cat)

In [98]:
#преобразуем бинарные признаки
label_encoder = LabelEncoder()

for column in bin:
    data_copy[column] = label_encoder.fit_transform(data_copy[column])

In [99]:
data_copy['price'].isnull().sum()
data_copy['price'].mean()

In [100]:
data_copy['price'] = data_copy['price'].fillna(1610198)

# ML

In [101]:
# Фиксируем RANDOM_SEED, чтобы наши эксперименты были воспроизводимы:
RANDOM_SEED = 42
VAL_SIZE = 0.2

In [102]:
X = data_copy.query('sample == 1').drop(['sample','price'], axis=1).values
X_sub = data_copy.query('sample == 0').drop(['sample','price'], axis=1).values

In [103]:
y = data_copy.query('sample == 1')['price'].values

In [104]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, 
                                                    shuffle=True, random_state=RANDOM_SEED)

## Model 1 LinearRegression

In [105]:
start = datetime.now()

naive = LinearRegression().fit(X_train, y_train/1.08)
y_pred = naive.predict(X_test)

In [106]:
# Напишем функцию для подсчёта метрики MAPE:
def mape(y_test, y_pred):
    return np.mean(np.abs((y_test - y_pred) / y_test))

# Напишем функцию для отображения метрики MAPE и времени, затраченного на обучение:

def print_learn_report(start, y_test, y_pred):
    print('\nВремя выполнения - ', datetime.now() - start)
    print(f"Точность по метрике MAPE:{(mape(y_test, y_pred))*100:0.2f}%")

In [107]:
# оцениваем точность
#predict = native.predict(X_test)
#print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")
print_learn_report(start, y_test, y_pred)

## Model 2 Cat Boost

In [108]:
cb = CatBoostRegressor(iterations = 5000,
                       random_seed = RANDOM_SEED,
                       eval_metric='MAPE',
                       custom_metric=['R2', 'MAE'],
                       silent=True)

In [109]:
start = datetime.now()

cb.fit(X_train, y_train,
             eval_set=(X_test, y_test),
             verbose_eval=0,
             use_best_model=True)

y_pred = cb.predict(X_test)

cb.save_model('catboost_single_model_baseline.model')
predict_submission = np.exp(cb.predict(X_sub))
print_learn_report(start, y_test, y_pred)

прологорифмирует y_train

In [110]:
cb = CatBoostRegressor(iterations = 5000,
                       random_seed = RANDOM_SEED,
                       eval_metric='MAPE',
                       custom_metric=['R2', 'MAE'],
                       silent=True)

In [111]:
start = datetime.now()

cb.fit(X_train, np.log(y_train),
             eval_set=(X_test, np.log(y_test)),
             verbose_eval=0,
             use_best_model=True)

y_pred = np.exp(cb.predict(X_test))

cb.save_model('catboost_single_model_baseline.model')
predict_submission = np.exp(cb.predict(X_sub))
print_learn_report(start, y_test, y_pred)

## Model 3 Gradient Boost

In [112]:
gb = GradientBoostingRegressor(min_samples_split=2, 
                               learning_rate=0.03, 
                               max_depth=10, 
                               n_estimators=1000)

In [113]:
start = datetime.now()

gb.fit(X_train, y_train)

#y_pred = gb.predict(X_test)
predict_submission = gb.predict(X_sub)
print_learn_report(start, y_test, y_pred)

In [114]:
# После логарифмирования

start = datetime.now()

gb.fit(X_train, np.log(y_train))

y_pred = gb.predict(X_test)
predict_submission = np.exp(gb.predict(X_sub))
print_learn_report(start, y_test, y_pred)

## Model 4 Random Forest

In [115]:
rf = RandomForestRegressor(n_estimators=1000, 
                            n_jobs=-1, 
                            max_depth=15, 
                            max_features='log2', 
                            random_state=RANDOM_SEED, 
                            oob_score=True)  

In [116]:
start = datetime.now()

rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
predict_submission = np.exp(rf.predict(X_sub))
print_learn_report(start, y_test, y_pred)

## Model 5 Xgboosting

In [117]:
import xgboost as xgb
xb = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5, learning_rate=0.03, \
                      max_depth=12, alpha=1, n_jobs=-1, n_estimators=1000)
xb.fit(X_train, np.log(y_train+1))
predict_submission = np.exp(xb.predict(X_sub))
print(f"Точность модели по метрике MAPE: {(mape(y_test, np.exp(xb.predict(X_test))))*100:0.2f}%")

# Stacking. Ansambles of models

In [118]:
# По причине долгих вычислений на kaggle, пришлось ограничить число деревьев в каждом методе до 100.
estimators = [
              ('lgbmr', LGBMRegressor(objective='regression', max_depth=12, num_leaves=1000,
                                      learning_rate=0.02, n_estimators=100, metric='mape',
                                      feature_fraction=0.6,)),
              ('gbr', GradientBoostingRegressor(min_samples_split=2,
                                                learning_rate=0.03,
                                                max_depth=10,
                                                n_estimators=100))]

st_ensemble = StackingRegressor(estimators=estimators,
                                final_estimator=LinearRegression())

# оцениваем точность
st_ensemble.fit(X_train, np.log(y_train))
predict_e = np.exp(st_ensemble.predict(X_test))
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_e))*100:0.2f}%")

In [119]:
predict_ensemble = np.exp(xb.predict(X_test))
print_learn_report(start, y_test, predict_ensemble)

# Submission

In [120]:
sample_submission['price'] = predict_submission
sample_submission.to_csv(f'submission_1.csv', index=False)
sample_submission.head()

Наилучший результат модели получился XGBoost. 
Для улучшения результатов модели можно было попробовать сгенерировать новые признаки, также не учитывалась разница в цене, т.к. парсились данные на август - сентябрь 2021г., т.е. инфляция, можно попробовать поформировать разные ансамбли моделей, также попробовать подобрать оптимальные параметры, но не было времени с этим разобраться, очень много времени потерял в парсингом данных, а потом еще с их чисткой и обработкой. 
Код по парсингу данных сделал в отдельном notebook, опубликовал на git